In [ ]:
%%capture
# output is suppressed but normally would spew out all the edc loading messages

import os
from pathlib import Path
from datetime import datetime
import pandas as pd
import numpy as np
import math
# import matplotlxib.pyplot as plt
# import seaborn as sns
import scipy.stats as stats

from dj_notebook import activate

env_file = os.environ["META_ENV"]
documents_folder = os.environ["META_DOCUMENTS_FOLDER"]
report_folder = Path(documents_folder)

plus = activate(dotenv_file=env_file)


In [ ]:
import itertools
from meta_analytics.dataframes import GlucoseEndpointsByDate, get_eos_df, get_screening_df
from meta_analytics.dataframes.screening import get_glucose_tested_only_df


In [ ]:
df_glu2 = get_glucose_tested_only_df()
cond_fasting2 = (df_glu2.fasting == "Yes") & (df_glu2.fasting_fbg_hrs >= 8.0)
cond_f2 = (df_glu2["gender"] == "F")
cond_m2 = (df_glu2["gender"] == "M")


In [ ]:
# df_glu[cond_f & cond_fasting & (df_glu.ogtt.notna()) & ((df_glu.fbg>=7.0) | (df_glu.ogtt>=11.1))].count()

# when ogtt not done 
# df_glu[cond_f & cond_fasting & (df_glu.ogtt.isna())].fbg.describe()

# we never have ogtt w/o fbg
# df_glu[cond_f & cond_fasting & (df_glu.fbg.isna())].ogtt.describe()

# df_glu = df_glu.reset_index(drop=False)
def dx(row):
    # print((row.fbg>=7.0) & (row.ogtt>=11.1))
    # print(row.fbg, row.ogtt)
    if (row.fbg>=7.0) & (row.fbg<10.0) & (row.ogtt>=11.1):
        ret = "fbg_ogtt"
    elif (row.fbg>=7.0) & (row.fbg<10.0) & (row.ogtt<11.1):
        ret = "fbg_only"
    elif (row.fbg<7.0) & (row.ogtt>=11.1):
        ret = "ogtt_only"
    elif (row.fbg<7.0) & (row.ogtt<11.1):
        ret = "neither"
    else:
        ret = None
        # print(row.fbg, row.ogtt)
    return ret
    
df_glu2["glucose"] = ""        
df_glu2["glucose"] = df_glu2[cond_fasting2].apply(lambda r: dx(r), axis=1)
# df_glu[cond_fasting & cond_f]


In [ ]:
df_glu2["glucose"].value_counts()


In [ ]:
df_glu2[cond_fasting2 & cond_f2]["glucose"].value_counts()


In [ ]:
df_glu2[cond_fasting2 & cond_m2]["glucose"].value_counts()

In [ ]:
from sklearn.metrics import cohen_kappa_score

In [ ]:
df_glu2[cond_fasting2 & cond_m2 & (df_glu2.fbg>=7.0) & (df_glu2.fbg<10.0) ].glucose.value_counts()
# y2 = df_glu2[cond_fasting2 & cond_f2 & (df_glu2.fbg<7.0)].glucose

In [ ]:
# cohen_kappa_score(y1, y2)

In [ ]:
cond_fasting = (df_glu2.fasting == "Yes") & (df_glu2.fasting_fbg_hrs >= 8.0)

df_glu2["fbg_threshold"] = (df_glu2[cond_fasting].fbg >= 7.0) & (df_glu2[cond_fasting].fbg < 10.0)
df_glu2["ogtt_threshold"] = df_glu2[cond_fasting].ogtt >= 11.1


In [ ]:
df_glu2["fbg_threshold"].value_counts()

In [ ]:
df_glu2["ogtt_threshold"].value_counts()

In [ ]:
# female
df_glu2[cond_f2][["fbg_threshold", "ogtt_threshold"]].value_counts()

In [ ]:
# male
df_glu2[cond_m2][["fbg_threshold", "ogtt_threshold"]].value_counts()

In [ ]:
from scipy.stats.contingency import odds_ratio

# female
res = odds_ratio([[76, 414], [72,3639]])
res.statistic

In [ ]:
res.confidence_interval(confidence_level=0.95)

In [ ]:
# male
res = odds_ratio([[28, 141], [48,1197]])
res.statistic

In [ ]:
res.confidence_interval(confidence_level=0.95)

In [ ]:
df_glu2[cond_fasting].count()